In [1]:
from transformers import BertModel, BertTokenizer, logging
import torch
import numpy as np
import re
import pandas as pd
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
#model.to(device)
device

'cuda'

In [ ]:
# save the results from running embeddings on colab
import pickle
with open('/content/drive/MyDrive/MLfile/train_embedding.pkl', 'wb') as f:
    pickle.dump(np.array(X_train_emb['embedding']), f)

with open('/content/drive/MyDrive/MLfile/train_label.pkl', 'wb') as f:
    pickle.dump(np.array(X_train_emb['label']), f)

import pickle
with open('/content/drive/MyDrive/MLfile/test_embedding.pkl', 'wb') as f:
    pickle.dump(np.array(X_test_emb['embedding']), f)

with open('/content/drive/MyDrive/MLfile/test_label.pkl', 'wb') as f:
    pickle.dump(np.array(X_test_emb['label']), f)

In [4]:
import pickle
#path = '/content/drive/MyDrive/MLfile/'
path ='embedding/'
with open(path + 'train_embedding.pkl', 'rb') as f:
    X = pickle.load(f)

with open(path +'train_label.pkl', 'rb') as f:
    y = pickle.load(f)

print(y.shape,X.shape)
X.shape

(25000,) (25000, 768)


(25000, 768)

In [5]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [6]:
X_train = np.array(X)[0:20000]
X_train.shape
y_train = np.array(y)[0:20000]
y_train.shape

(20000,)

In [7]:
X_valid = np.array(X)[20000:]
y_valid = np.array(y)[20000:]
X_valid.shape

(5000, 768)

In [8]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [19]:
epochs = 100000
input_dim = 768 # dimension of input vectors
output_dim = 1 # Single binary output 
learning_rate = 0.01 # todo: change learning rate to inverse sqrt

In [10]:
model = LogisticRegression(input_dim,output_dim)
criterion = torch.nn.BCELoss()

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
X_train, X_valid = torch.Tensor(X_train),torch.Tensor(X_valid)
y_train, y_valid = torch.Tensor(y_train),torch.Tensor(y_valid)

In [16]:
from tqdm import tqdm

In [22]:
losses = []
losses_valid = []
Iterations = []
iter = 0
for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
    x = X_train
    labels = y_train
    optimizer.zero_grad() # Setting our stored gradients equal to zero
    outputs = model(X_train)
    loss = criterion(torch.squeeze(outputs), labels) 
    
    loss.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
    
    optimizer.step() # Updates weights and biases with the optimizer (SGD)
    
    iter+=1
    if iter%10000==0:
        with torch.no_grad():
            # Calculating the loss and accuracy for the test dataset
            correct_valid = 0
            total_valid = 0
            outputs_valid = torch.squeeze(model(X_valid))
            loss_valid = criterion(outputs_valid, y_valid)
            
            predicted_valid = outputs_valid.round().detach().numpy()
            total_valid += y_valid.size(0)
            correct_valid += np.sum(predicted_valid == y_valid.detach().numpy())
            accuracy_valid = 100 * correct_valid/total_valid
            losses_valid.append(loss_valid.item())
            
            # Calculating the loss and accuracy for the train dataset
            total = 0
            correct = 0
            total += y_train.size(0)
            correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == y_train.detach().numpy())
            accuracy = 100 * correct/total
            losses.append(loss.item())
            Iterations.append(iter)
            
            print(f"Iteration: {iter}. \nvalid - Loss: {loss_valid.item()}. Accuracy: {accuracy_valid}")
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")

Training Epochs:  10%|█████▋                                                   | 10037/100000 [00:37<05:34, 268.83it/s]

Iteration: 10000. 
valid - Loss: 0.311367005109787. Accuracy: 88.0
Train -  Loss: 0.2752290368080139. Accuracy: 89.355



Training Epochs:  20%|███████████▍                                             | 20053/100000 [01:15<04:55, 270.27it/s]

Iteration: 20000. 
valid - Loss: 0.3112168312072754. Accuracy: 88.04
Train -  Loss: 0.27440571784973145. Accuracy: 89.425



Training Epochs:  30%|█████████████████▏                                       | 30046/100000 [01:52<04:19, 269.94it/s]

...

Iteration: 80000. 
valid - Loss: 0.31108975410461426. Accuracy: 88.08
Train -  Loss: 0.2707759141921997. Accuracy: 89.67



Training Epochs:  90%|███████████████████████████████████████████████████▎     | 90031/100000 [05:38<00:38, 261.75it/s]

Iteration: 90000. 
valid - Loss: 0.3111453056335449. Accuracy: 88.06
Train -  Loss: 0.2703252136707306. Accuracy: 89.69



Training Epochs: 100%|████████████████████████████████████████████████████████| 100000/100000 [06:15<00:00, 266.14it/s]

Iteration: 100000. 
valid - Loss: 0.3112138509750366. Accuracy: 88.04
Train -  Loss: 0.2699064016342163. Accuracy: 89.7



In [47]:
def print_error(clf):
    print('training error is', np.sum(abs(clf.predict(X_train) - y_train)) / len(y_train))
    print('validation error is', np.sum(abs(clf.predict(X_valid) - y_valid)) / len(y_valid))
    

In [50]:
# logistic regression with L1 and L2 regularization
logistic_clf = LogisticRegression(random_state=0, max_iter = 100000, solver = 'saga', penalty = 'elasticnet', l1_ratio = 0.5, C = 10).fit(X_train, y_train)

In [51]:
print_error(logistic_clf)

training error is 0.1021
validation error is 0.123


In [52]:
from sklearn import svm
svm_clf = svm.SVC(kernel='rbf')
svm_clf.fit(X_train, y_train)

SVC()

In [53]:
print_error(svm_clf)

training error is 0.1335
validation error is 0.1434


In [ ]:
from sklearn.model_selection import GridSearchCV

logistic_gs = GridSearchCV(
    LogisticRegression(random_state=0, max_iter = 100000, solver = 'saga', penalty = 'elasticnet', C = 1),
    param_grid={"l1_ratio": np.linspace(0,1,11)},
    #scoring=scoring,
    refit="AUC",
    return_train_score=True,
)
logistic_gs.fit(X_train, y_train)

In [58]:
np.linspace(0,1,11)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [ ]:
from sklearn.model_selection import GridSearchCV

svm_gs = GridSearchCV(
    svm.SVC(random_state=0, kernel = 'poly'),
    param_grid={"degree": range(1, 10, 1)},
    #scoring=scoring,
    refit="AUC",
    return_train_score=True,
)
svm_gs.fit(X_train, y_train)

In [ ]:
print_error(logistic_gs)
print_error(svm_clf)